Import libraries and extensions

In [1]:
import tensorflow as tf

config = tf.compat.v1.ConfigProto(gpu_options=tf.compat.v1.GPUOptions(allow_growth=True))
sess = tf.compat.v1.Session(config=config)
print(tf.__version__)
print('Num GPUs available: ', len(tf.config.list_physical_devices('GPU')))

import numpy as np
import os
import pickle

from tensorflow import keras
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D
from tensorflow.keras.layers import Dense,GlobalMaxPooling2D,MaxPool2D
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers

2.5.0
Num GPUs available:  0


Import the model to be validated, as well as the respective institution's validation dataset. Change the variables as shown in the commented lines below to perform internal or external validation on the global model with this code.

In [23]:
# global_weights_dir = r'C:\Documents\iter_Y\Global_X.h5'
instX_weights_dir = r'C:\Documents\iter_Y\weights_directory\Institution_X.h5'

# data_dir =  r'C:\Documents\Institution_1'
data_dir = r'C:\Documents\Institution_X'
val_dir = os.path.join(data_dir,'valid_seq')

batch_size = 32
input_shape = (224, 224, 3)

valid_datagen = ImageDataGenerator(rescale=1./255)

valid_generator = valid_datagen.flow_from_directory(val_dir, class_mode='binary', batch_size=batch_size,target_size=(224, 224))


Found 10790 images belonging to 2 classes.


Function to construct a model of the appropriate shape.

In [4]:
batch_size = 32
input_shape = (224, 224, 3)

def create_model(input_shape):

    base_model=VGG16(weights=None,include_top=False,input_shape=input_shape)

    x=base_model.output
    x=keras.layers.GlobalMaxPooling2D()(x)
    x=keras.layers.Dense(1024,activation='relu')(x)
    x=keras.layers.Dense(1024,activation='relu')(x)
    x=keras.layers.Dropout(0.25)(x)
    x=keras.layers.Dense(1024,activation='relu')(x)
    x=keras.layers.Dense(512,activation='relu')(x)
    preds=keras.layers.Dense(1,activation='sigmoid')(x)
    
    model=keras.Model(inputs=base_model.input,outputs=preds)
    model.trainable = True

    for layer in model.layers[:16]:
        layer.trainable=False
    for layer in model.layers[16:]:
        layer.trainable=True

    return model

Compile the model.

In [8]:
model = create_model(input_shape)
model.compile(loss='binary_crossentropy', metrics=['accuracy',keras.metrics.Recall(),keras.metrics.Precision()])

Load the institution's weights into the model that was just created.

In [13]:
model.load_weights(instX_weights_dir)

Call the evaluate method on the model to validate it, passing the validation dataset as a parameter.

In [28]:
model.evaluate(valid_generator)

338/338 [==============================] - 1018s 3s/step - loss: 0.4780 - accuracy: 0.7405 - recall_2: 0.1251 - precision_2: 0.1027


[0.47800716757774353,
 0.7405004501342773,
 0.12508784234523773,
 0.10271205753087997]